In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import inception_blocks_v4
import inception_blocks_v4_modified
import glob
import PIL
#import inception.preprocessing
import matplotlib.image as mpimg
#from data_aug import *
import os
#from tensorflow import data
from utils import *
slim = tf.contrib.slim

#Pretrained_weights = "/checkpoint_files/inception_v4.ckpt" 3453
ckpt_dir = "checkpoint_files/inception_v4.ckpt"
new_ckpt_dir = "checkpoints_files/model.ckpt"
image_size = 299
images_file_path = "C:/Users/ashus/Dog_classification/output/*.jpg"
labels_dir = "labels.csv"
PATH = "C:/Users/ashus/Dog_classification/output"

pre_num_classes = 1001
num_classes = 120
iterations = 100

c:\users\ashus\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with tf.device('/device:CPU:0'):
    #with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    with slim.arg_scope(inception_blocks_v4.inception_v4_arg_scope()):
        X_input = tf.placeholder(tf.float32, shape = (None, image_size, image_size, 3))
        Y_label = tf.placeholder(tf.float32, shape = (None, num_classes))
            
        targets = convert_to_onehot(labels_dir, no_of_features = num_classes)
        assert targets.shape[1] == 120, 'THE TARGETS SHAPE IS NOT CORRECT'
        targets = tf.constant(targets, dtype = tf.float32)

        Images = [] #TO STORE THE RESIZED IMAGES IN THE FORM OF LIST TO PASS IT TO tf.train.batch()
        images = glob.glob(images_file_path)
        i = 0
        for my_img in images:
            image = mpimg.imread(my_img)[:, :, :3]
            #print (image.shape)
            #image = np.asarray(image, dtype = np.float32)
            image = tf.constant(image, dtype = tf.float32)
            Images.append(image)
            i = i + 1
            if i == 1:
                break

        #try:
        #pretrained_weights = slim.assign_from_checkpoint_fn(ckpt_dir, slim.get_model_variables('InceptionV4'))
        #with tf.Session() as sess:
         #   pretrained_weights(sess)
        #except ValueError:
         #   print("The checkpoint file has some error")

        logits, end_points = inception_blocks_v4.inception_v4(inputs = X_input, num_classes = pre_num_classes, is_training = True, create_aux_logits= False)
        pretrained_weights = slim.assign_from_checkpoint_fn(ckpt_dir, slim.get_model_variables('InceptionV4'))
        with tf.Session() as sess:
            pretrained_weights(sess)
        
        #MY LAYERS, add bias as well
        my_layer = slim.fully_connected(logits, 560, activation_fn=tf.nn.relu, scope=None, weights_initializer = tf.truncated_normal_initializer(stddev = 0.001), weights_regularizer=slim.l2_regularizer(0.00005),biases_initializer = tf.truncated_normal_initializer(stddev=0.001), biases_regularizer=slim.l2_regularizer(0.00005))
        my_layer = slim.dropout(my_layer, keep_prob = 0.6, scope = None)
        my_layer = slim.fully_connected(my_layer, num_classes,activation_fn = tf.nn.relu,scope= None, weights_initializer = tf.truncated_normal_initializer(stddev=0.001), weights_regularizer=slim.l2_regularizer(0.00005), biases_initializer = tf.truncated_normal_initializer(stddev=0.001), biases_regularizer=slim.l2_regularizer(0.00005))
        my_layer_logits = slim.fully_connected(my_layer, num_classes, activation_fn=None,scope=None)

        #loss = tf.losses.mean_squared_error(labels = Y_label, predictions = predictions)
        #predictions = tf.nn.softmax(logits)

        # Add the loss function to the graph.   
        loss = tf.losses.softmax_cross_entropy(onehot_labels = Y_label, logits = my_layer_logits)  
        #onehot_labels are the actual labels and logits are the predictions, we don't need to take the softmax of labels
        # The total loss is the user's loss plus any regularization losses. slim.losses.get_total_loss() is deprecated
        # to calc regularization losses use tf.losses.get_regularization_losses
        #total_loss = tf.losses.get_total_loss()

        # Specify the optimizer and create the train op:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = slim.learning.create_train_op(loss, optimizer) 

        #Generating batch
        
        batch_size = 1
        images, labels = tf.train.batch([Images, targets], batch_size = 1, num_threads = 1, capacity = batch_size, enqueue_many=True)
        tensor_images = tf.convert_to_tensor(images, dtype = tf.float32)
        tensor_labels = tf.convert_to_tensor(labels, dtype = tf.float32)
        #This function is generating (8,8,299,299,3). The first is my batch size
        
        #the below 2 lines is for countering images.shape =(8, 1, 299, 299, 3) problem
        #images = tf.reshape(images, shape = (8, 299, 299, 3))
        #labels = tf.reshape(labels, shape = (8, 120))
    
        with tf.Session() as sess:
            print (tensor_images)
            print (tensor_labels)
        #images, labels = load_batch(8, label = targets)
        # Run the training inside a session.
        final_loss = slim.learning.train(train_op,logdir = new_ckpt_dir, number_of_steps = iterations, save_summaries_secs=5,log_every_n_steps=50)(feed_dict = {X_input:tensor_images ,Y_label: tensor_labels})  #{X_input:images ,Y_label: labels}


INFO:tensorflow:Restoring parameters from checkpoint_files/inception_v4.ckpt
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Tensor("batch:0", shape=(1, 299, 299, 3), dtype=float32, device=/device:CPU:0)
Tensor("batch:1", shape=(1, 120), dtype=float32, device=/device:CPU:0)
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from checkpoints_files/model.ckpt\model.ckpt-0
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path checkpoints_files/model.ckpt\model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,299,299,3]
	 [[Node:

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,299,299,3]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,299,299,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "c:\users\ashus\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\ashus\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\ashus\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\ashus\anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\ashus\anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "c:\users\ashus\anaconda3\envs\tenso